# Redes Neuronales y Funciones de Activación

José Julián Camacho Hernández


In [2]:
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### 1. Verificando el comportamiento de las funciones de activación

In [3]:
def computeMetrics(y_test, y_pred, training_time):
    acc = accuracy_score(y_test, y_pred)                        # Calcular la exactitud
    f1 = f1_score(y_test, y_pred, average='macro')              # Calcular F1 score
    rec = recall_score(y_test, y_pred, average='macro')         # Calcular el recall
    prec = precision_score(y_test, y_pred, average='macro')     # Calcular la precisión
    metrics = {"Accuracy":acc, "Precision":prec, "Recall":rec, "F1 Score":f1, "Tiempo de entrenamiento [s]":training_time}
    df = pd.DataFrame(metrics, index = [0])
    display(df)

In [6]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    #def activacion(self, x):
    #    # Función de activación sigmoide
    #    return 1.0 / (1 + np.exp(-x))
    
    #def activacion(self, x):
    #    # Función de activación tanh
    #    return np.tanh(x)
    
    def activacion(self, x):
        # Función de activación ReLU
        return np.maximum(0.0001*x, x)

    #def activacion_derivada(self, x):
    #    # Derivada de la función de activación sigmoide
    #    return x * (1 - x)
    
    #def activacion_derivada(self, x):
    #    # Derivada de la función de activación tanh
    #    return (1 - x**2)
        
    def activacion_derivada(self, x):
        # Derivada de la función de activación ReLU
        return np.where(x > 0, 1, 0.0001)
        
    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x)
            #print('y: ', y)
            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.activacion_derivada(capa_activacion[-1])
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
            delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)

    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

In [4]:
# Cargar el conjunto de datos Iris
iris = load_iris()
X = iris.data
y = iris.target

mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### 1.1. Ejecución con un α = 0,15 y con epochs = 3000

In [9]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15)

#Tomar tiempo de entrenamiento
start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()
time_taken = end_time - start_time

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
computeMetrics(y_prueba, predicciones, time_taken)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,1.0,1.0,1.0,1.0,19.484454


Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 1.0


#### 1.2. Resultados para tanh

In [48]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15)

#Tomar tiempo de entrenamiento
start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()
time_taken = end_time - start_time

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
computeMetrics(y_prueba, predicciones, time_taken)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,0.866667,0.88141,0.866667,0.864904,15.930967


Predicciones:  [1, 2, 1, 1, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.8666666666666667


#### 1.2. Resultados para ReLU

In [22]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.15)

#Tomar tiempo de entrenamiento
start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3000)
end_time = time.time()
time_taken = end_time - start_time

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
computeMetrics(y_prueba, predicciones, time_taken)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

/tmp/ipykernel_12107/4194306296.py:59: RuntimeWarning: invalid value encountered in subtract
  self.pesos[i] -= self.alpha * d_peso
/tmp/ipykernel_12107/4194306296.py:60: RuntimeWarning: invalid value encountered in subtract
  self.bias[i] -= self.alpha * d_bias
/home/jose/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,0.333333,0.111111,0.333333,0.166667,19.415446


Predicciones:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.3333333333333333


### 1.3. Ajuste de los parámetros α y epochs

#### 1.3.1. Ajuste para tanh: α = 0,125 y con epochs = 3500

In [61]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.125)

#Tomar tiempo de entrenamiento
start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=3500)
end_time = time.time()
time_taken = end_time - start_time

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
computeMetrics(y_prueba, predicciones, time_taken)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,1.0,1.0,1.0,1.0,19.153458


Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 1.0


#### 1.3.2. Ajuste para ReLU: α = 0,125 y con epochs = 100

In [63]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[4,4,4,3], alpha=0.125)

#Tomar tiempo de entrenamiento
start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=100)
end_time = time.time()
time_taken = end_time - start_time

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
computeMetrics(y_prueba, predicciones, time_taken)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,0.9,0.923077,0.9,0.897698,0.707778


Predicciones:  [0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 0, 1, 2, 0, 2, 1, 2, 2, 1, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.9


#### 1.4. ¿Cuál función de activación arrojó los mejores resultados?

Para los parámetros iniciales la función sigmoid arrojó los mejores resultados al predecir correctamente todos los casos. 

Al ajutar los parámetros, el mlp con función tanh mejoró y también logró predecir de manera correcta y así tener métricas perfectas.

ReLU en un inicio no logró predecir correctamente, pero se bajaron los epochs para evitar resultados NaN y se lograron métricas mucho mejores.

### MLP con Sklearn

In [25]:
# Definir modelo
mlp = MLPClassifier()

# Parámetros
param_grid = {
    'hidden_layer_sizes': [(5,), (10,), (15,)],
    'activation': ['sigmoid', 'tanh', 'relu'],
    'alpha': np.logspace(-4, 0, 5),
    'max_iter': [200, 250, 300, 400]
}

# Usar GridSearch para encontrar los valores óptimos
grid_search = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_entrenamiento, y_entrenamiento)
print("Optimal alpha:", grid_search.best_params_['alpha'])
print("Optimal epochs:", grid_search.best_params_['max_iter'])
print("Optimal activation:", grid_search.best_params_['activation'])
print("Optimal hidden_layer_sizes:", grid_search.best_params_['hidden_layer_sizes'])

alpha = grid_search.best_params_['alpha']
epochs = grid_search.best_params_['max_iter']
activation = grid_search.best_params_['activation']
hidden_layer_sizes = grid_search.best_params_['hidden_layer_sizes']

# Crear nuevo MLP con los mejores parámetros
best_mlp = MLPClassifier(alpha=alpha, max_iter=epochs, activation=activation, 
                    hidden_layer_sizes=hidden_layer_sizes)

# Tomar tiempo de entrenamiento
start_time = time.time()
best_mlp.fit(X_entrenamiento, y_entrenamiento)
end_time = time.time()
time_taken = end_time - start_time

# Predecir
y_pred = best_mlp.predict(X_prueba)

computeMetrics(y_prueba, y_pred, time_taken)

/home/jose/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/jose/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/jose/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/jose/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/jose/.local/lib/python3.10/site-packages/sklearn/n

Optimal alpha: 0.1
Optimal epochs: 300
Optimal activation: relu
Optimal hidden_layer_sizes: (15,)


/home/jose/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/jose/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,0.9,0.902357,0.9,0.899749,0.141973


Para Sklearn se obtiene que la mejor función de activación para este caso es relu con 300 epochs y alpha de 0.1
Se nota que el tiempo de entrenamiento de Sklearn es efectivamente menor a las implementaciones propias.

### 2. Creando una red específica

#### 2.1. Modificación de la red para que pueda tener capas ocultas de diferentes tamaños.

In [14]:
class Perceptron_Multicapa:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)
        #print(self.pesos)
        #print(self.bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            #print('x', x)
            y = self.activacion(x)
            capa_activacion.append(y)
        #print('capa activacion forward', capa_activacion)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.activacion_derivada(capa_activacion[-1])
        #print('delta', delta)

        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            #print('activacion_actual', activacion_actual)
            activacion_anterior = capa_activacion[i-1] if i > 0 else X
            #print('activacion_anterior', activacion_anterior)
            d_peso = np.outer(activacion_actual, delta)
            d_bias = delta
            #print('self.pesos[i]', self.pesos[i])
            #print('d_peso', d_peso)
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
            delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)
            #print('delta', delta)

    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

#### 2.2. Cree una red neuronal y obtenga las métricas

In [20]:
# Crear y entrenar el perceptrón multicapa
perceptron = Perceptron_Multicapa(capas=[4,8,5,3], alpha=0.15)

#Tomar tiempo de entrenamiento
start_time = time.time()
#print(np.eye(3)[y_entrenamiento])
perceptron.entrenar(X_entrenamiento, np.eye(3)[y_entrenamiento], epochs=2500)
end_time = time.time()
time_taken = end_time - start_time

# Hacer predicciones sobre el conjunto de prueba
predicciones = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    predicciones.append(prediccion_clase)

# Calcular la precisión de las predicciones
computeMetrics(y_prueba, predicciones, time_taken)

# Calcular la precisión de las predicciones
print("Predicciones: ", predicciones)
print("Y_Real      : ", y_prueba)
precision = sum(predicciones == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento [s]
0,0.966667,0.969697,0.966667,0.966583,28.027327


Predicciones:  [0, 2, 1, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 2, 1, 0, 2, 0]
Y_Real      :  [0 2 1 1 0 1 0 0 2 1 2 2 2 1 0 0 0 1 1 2 0 2 1 2 2 1 1 0 2 0]
Precisión: 0.9666666666666667
